# Time-domain ODMR measurement (with APD & Time Tagger)

![frequency-shceduling](./figure/time-scheduling.jpg)


Import necessary modules and functions firstly

In [1]:
import matplotlib.pyplot as plt
from odmactor.scheduler import RamseyScheduler, RabiScheduler, RelaxationScheduler
from odmactor.scheduler import HahnEchoScheduler, HighDecouplingScheduler
from odmactor.utils.plotting import plot_ramsey, plot_rabi, plot_t1
import scipy.constants as C
import numpy as np


## Ramsey detecting

**1. Set measurement parameters**
- number of readout operation $N$ per time interval measurement point
- delay time range for scanning modes, i.e., `[t_start, t_end, t_step]`, unit: "ns"
- calibrated MW $\pi$ pulse parameters, i.e., frequency (unit: Hz), power (unit: dBm) and time (unit: s)
- laser initialization time $t_{init}$, signal readout pulse time $t_{sig}$, etc.
- optional parameters: MW power $p$, if designed, the time of built-in MW $\pi$ pulse will be regulated correspondingly
- time bin $t$ per readout operation will be calculated dynamically when running the scheduler on scanning-time mode

In [ ]:

t_start = 40
t_end = 7000
t_step = 30

pi_power = 12
pi_freq = 2.852 * C.giga
pi_time = 310 * C.nano

t_init = 5e3
inter_init_mw = 3e3
inter_mw_read = 200
pre_read = 50
t_read_sig = 800
# t_read_ref = t_read_sig

N = int(1e6 / 4)

**2. Construct a Ramsey Scheduler and run**

Especially, the calibrated MW $\pi$ pulse should be configured


In [ ]:
scheduler = RamseyScheduler(mw_ttl=1, with_ref=True, epoch_omit=5)
scheduler.configure_mw_paras(power=pi_power, freq=pi_freq)
scheduler.pi_pulse['freq'], scheduler.pi_pulse['power'], scheduler.pi_pulse['time'] = pi_freq, pi_power, pi_time
scheduler.configure_odmr_seq(t_init, t_read_sig, inter_init_mw=inter_init_mw, inter_mw_read=inter_mw_read,
                             pre_read=pre_read, N=N)
scheduler.set_delay_times(t_start, t_end, t_step)
scheduler.configure_tagger_counting(reader='cbm')

In [ ]:
scheduler.run_scanning()
scheduler.close()

In [ ]:
# observe detection sequence
scheduler._gene_pseudo_detect_seq()
fig = scheduler.sequences_figure

**3. Calculate contrast and plot figures**

In [ ]:
## count & contrast
counts_sig_ref = scheduler.result  # [times, counts, counts_ref]
contrast = [sig / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]
contrast_2 = [abs(sig - ref) / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]


In [ ]:
# plot_ramsey(counts_sig_ref[0], contrast)

plt.style.use('seaborn')
plt.plot(counts_sig_ref[0], counts_sig_ref[1], 'o-')
plt.plot(counts_sig_ref[0], counts_sig_ref[2], 'o-')
plt.title('Ramsey')
plt.xlabel('Duration time (ns)')
plt.ylabel('Count')


In [ ]:
plt.plot(counts_sig_ref[0][1:], contrast[1:], 'o-')
plt.title('Ramsey')

In [ ]:
plt.plot(counts_sig_ref[0][1:], contrast_2[1:], 'o-')
plt.title('Ramsey contrast')

In [ ]:
np.savetxt('ramsey_2.852ghz_310ns.txt', scheduler.result)

## Rabi oscillation

**1. Set measurement parameters**
- number of readout operation $N$ per time interval measurement point
- delay time range for scanning modes, i.e., `[t_start, t_end, t_step]`, unit: "ns"
- laser initilization time $t_{init}$, singal readout pulse time $t_{sig}$, etc.
- MW power $p$, MW frequency $f$
- time bin $t$ per readout operation will be calculated dynamically when running the scheduler on scanning-time mode

In [ ]:

t_start = 20
t_end = 2000
t_step = 30

t_init = 3e3
inter_init_mw = 3000
inter_mw_read = 200

t_read_sig = 800
pre_read = 50

N = int(1e6 / 4)

p = 12
f = 2.889 * C.giga

**2. Construct a Rabi measurement Scheduler and run**

In [ ]:
scheduler = RabiScheduler(mw_ttl=1, with_ref=True, epoch_omit=5)
scheduler.configure_mw_paras(power=p, freq=f)
scheduler.configure_odmr_seq(t_init, t_read_sig, pre_read=pre_read, inter_init_mw=inter_init_mw,
                             inter_mw_read=inter_mw_read, N=N)
scheduler.set_delay_times(t_start, t_end, t_step)
scheduler.configure_tagger_counting(reader='cbm')


In [ ]:
scheduler.run_scanning()

scheduler.close()

In [ ]:
scheduler._gene_pseudo_detect_seq()
fig = scheduler.sequences_figure

**3. Calculate contrast and plot figures**

In [ ]:
## count & contrast
counts_sig_ref = scheduler.result  # [times, counts, counts_ref]
contrast = [sig / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]
contrast_2 = [abs(sig - ref) / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]

In [ ]:
# plot_rabi(counts_sig_ref[0], counts_sig_ref[1])
plt.style.use('seaborn')
#plt.vlines(210, 13.6,14)
plt.plot(counts_sig_ref[0], counts_sig_ref[1], 'o--', label='signal')
plt.plot(counts_sig_ref[0], counts_sig_ref[2], 'o-', label='referengce')
plt.legend()
plt.title('Rabi Oscillation')
plt.xlabel('MW time (ns)')
plt.ylabel('counts')

In [ ]:
plt.plot(counts_sig_ref[0][1:], contrast[1:], 'o-')

In [ ]:
plt.plot(counts_sig_ref[0][1:], contrast_2[1:], 'o-')

In [ ]:
np.savetxt('rabi_2.889ghz_12dbm_long_7us.txt', scheduler.result)


## T1 relaxometry

**1. Set measurement parameters**
- number of readout operation $N$ per time interval measurement point
- delay time range for scanning modes, i.e., `[t_start, t_end, t_step]`, unit: "ns"
- laser initialization time $t_{init}$, signal readout pulse time $t_{sig}$, etc.
- calibrated MW $\pi$ pulse parameters, i.e., frequency (unit: Hz), power (unit: dBm) and time (unit: s)
- optional parameters: MW power $p$, if designed, the time of built-in MW $\pi$ pulse will be regulated correspondingly
- time bin $t$ per readout operation will be calculated dynamically when running the scheduler on scanning-time mode

In [ ]:
t_start = 1e3
t_end = 1e6
# t_step = 1e4
t_length = 51

pi_power = 10
pi_freq = 2.852 * C.giga
pi_time = 210 * C.nano

t_init = 3e3
inter_init_mw = 1e3
t_read_sig = 800
t_read_ref = t_read_sig

N = int(1e6 / 4)

**2. Construct a T1 measurement scheduler and run**

In [ ]:
scheduler = RelaxationScheduler(mw_ttl=1, ms=0, epoch_omit=5)
scheduler.pi_pulse['freq'], scheduler.pi_pulse['power'], scheduler.pi_pulse['time'] = pi_freq, pi_power, pi_time
scheduler.configure_odmr_seq(t_init, t_read_sig, inter_init_mw=inter_init_mw, N=N)
scheduler.set_delay_times(t_start, t_end, length=t_length, logarithm=True)  # exponential step interval
scheduler.configure_tagger_counting(reader='cbm')

print('scheduler for detecting relaxation time of state Ms={}'.format(scheduler.ms))

In [ ]:
scheduler.run_scanning()
scheduler.close()


In [ ]:
# observe sequence figure
fig = scheduler.sequences_figure

**3. Calculate contrast and plot figures**

In [ ]:
## count & contrast
counts_sig_ref = scheduler.result  # [times, counts, counts_ref]
contrast = [sig / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]

In [ ]:
plot_t1(counts_sig_ref[0], contrast)



## Hahn echo measurement

**1. Set measurement parameters**
- number of readout operation $N$ per time interval point
- delay time range for scanning modes, i.e., `[t_start, t_end, t_step]`, unit: "ns"
- calibrated MW $\pi$ pulse parameters, i.e., frequency (unit: Hz), power (unit: dBm) and time (unit: s)
- laser initialization time $t_{init}$, signal readout pulse time $t_{sig}$, etc.
- optional parameters: MW power $p$, if designed, the time of built-in MW $\pi$ pulse will be regulated correspondingly
- time bin $t$ per readout operation will be calculated dynamically when running the scheduler on scanning-time mode

In [ ]:
t_start = 40
t_end = 7000
t_step = 30

pi_power = 12
pi_freq = 2.852 * C.giga
pi_time = 310 * C.nano

t_init = 5e3
inter_init_mw = 3e3
inter_mw_read = 200
pre_read = 50
t_read_sig = 800
# t_read_ref = t_read_sig

N = int(1e6 / 4)


**2. Construct a Hahn echo measurement Scheduler and run**

In [ ]:
scheduler = HahnEchoScheduler(mw_ttl=0, with_ref=True, epoch_omit=5)
scheduler.configure_mw_paras(power=pi_power, freq=pi_freq)
scheduler.pi_pulse['time'], scheduler.pi_pulse['freq'], scheduler.pi_pulse['power'] = pi_time, pi_freq, pi_power
scheduler.configure_odmr_seq(t_init, t_read_sig, inter_init_mw, inter_mw_read, pre_read, N=N)
scheduler.set_delay_times(t_start, t_end, t_step)
scheduler.configure_tagger_counting(reader='cbm')

In [ ]:
scheduler.run_scanning()
scheduler.close()

**3. Calculate contrast and plot figures**

In [ ]:
counts_sig_ref = scheduler.result  # [times, counts, counts_ref]
contrast = [sig / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]
contrast_2 = [abs(sig - ref) / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]

In [ ]:
# plot_rabi(counts_sig_ref[0], counts_sig_ref[1])
plt.style.use('seaborn')
#plt.vlines(210, 13.6,14)
plt.plot(counts_sig_ref[0], counts_sig_ref[1], 'o--', label='signal')
plt.plot(counts_sig_ref[0], counts_sig_ref[2], 'o-', label='referengce')
plt.legend()
plt.title('Hahn echo')
plt.xlabel('MW time (ns)')
plt.ylabel('counts')

In [ ]:
plt.plot(counts_sig_ref[0][1:], contrast[1:], 'o-')

In [ ]:
plt.plot(counts_sig_ref[0][1:], contrast_2[1:], 'o-')

## High-order dynamical decoupling

**1. Set measurement parameters**
- number of readout operation $N$ per time interval point
- dynamical decoupling order, i.e., $\pi$ pulse number
- delay time range for scanning modes, i.e., `[t_start, t_end, t_step]`, unit: "ns"
- calibrated MW $\pi$ pulse parameters, i.e., frequency (unit: Hz), power (unit: dBm) and time (unit: s)
- laser initialization time $t_{init}$, signal readout pulse time $t_{sig}$, etc.
- optional parameters: MW power $p$, if designed, the time of built-in MW $\pi$ pulse will be regulated correspondingly
- time bin $t$ per readout operation will be calculated dynamically when running the scheduler on scanning-time mode

In [ ]:
t_start = 40
t_end = 7000
t_step = 30

pi_power = 12
pi_freq = 2.852 * C.giga
pi_time = 310 * C.nano

t_init = 5e3
inter_init_mw = 3e3
inter_mw_read = 200
pre_read = 50
t_read_sig = 800
# t_read_ref = t_read_sig

N = int(1e6 / 4)
d = 8 # decoupling order


**2. Construct a high-order dynamic decoupling measurement Scheduler and run**

In [ ]:
scheduler = HighDecouplingScheduler(mw_ttl=0, with_ref=True, epoch_omit=5, order=d)
scheduler.configure_mw_paras(power=pi_power, freq=pi_freq)
scheduler.pi_pulse['time'], scheduler.pi_pulse['freq'], scheduler.pi_pulse['power'] = pi_time, pi_freq, pi_power
scheduler.configure_odmr_seq(t_init, t_read_sig, inter_init_mw, inter_mw_read, pre_read, N=N)
scheduler.set_delay_times(t_start, t_end, t_step)
scheduler.configure_tagger_counting(reader='cbm')

In [ ]:
scheduler.run_scanning()
scheduler.close()

**3. Calculate contrast and plot figures**


In [ ]:
counts_sig_ref = scheduler.result  # [times, counts, counts_ref]
contrast = [sig / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]
contrast_2 = [abs(sig - ref) / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]

In [ ]:
plt.plot(counts_sig_ref[0], counts_sig_ref[1], 'o--', label='signal')
plt.plot(counts_sig_ref[0], counts_sig_ref[2], 'o-', label='referengce')
plt.legend()
plt.title('High-order DD')
plt.xlabel('MW time (ns)')
plt.ylabel('counts')